# Preparing Snowflake
This note book will guide you through the required steps thats need to prepare your Snowflake account to deploy the demo [Streamlit ML App](https://github.com/kameshsampath/ks-st-mt-app)

Typically we wil 

- [x] Create Schemas and Tables
- [x] Create a external stage to S3
- [x] Load Penguins Dataset on the table

In [ ]:
-- data schema
create schema if not exists data;
-- create schema to hold all stages
create schema if not exists stages;
-- create schema to hold all file formats
create schema if not exists file_formats;
-- apps to hold all streamlit apps
create schema if not exists apps;

create stage if not exists stages.st_ml_app_penguins
  url='s3://'

create file format if not exist file_formats.csv
  type='csv'
  SKIP_HEADER=1;

In [ ]:
import pandas as pd

df = pd.read_csv('https://github.com/dataprofessor/data/blob/master/penguins_cleaned.csv')
df